# Synopsis

Here we create a TFIDF matrix from our corpus of novels, and then save a reduced version of this for use in HCA and PCA models. We choose to limit our vocabulary to 1000 top words, based on high-frequency non-stopwords.

We begin by extracting a bag-of-words from the token table. Note that we could have chosen a different set of "bags," e.g. paragraphs or an arbitrary chunk of n tokens.

# Configuration

In [14]:
db_name = "/sfs/qumulo/qhome/sk5be/DS5559/HarryPotter.db"

OHCO = ['book_num', 'chap_num', 'para_num', 'sent_num', 'token_num']
BOOKS = OHCO[:1]
CHAPS = OHCO[:2]
PARAS = OHCO[:3]
SENTS = OHCO[:4]

BAG = CHAPS
print(CHAPS)

['book_num', 'chap_num']


# Libraries

In [2]:
import sqlite3
import pandas as pd
import numpy as np

# Pragmas

In [3]:
%matplotlib inline

# Process

In [7]:
with sqlite3.connect(db_name) as db:
    tokens = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    vocab = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    docs =  pd.read_sql('SELECT * FROM doc', db, index_col=CHAPS)

## Create DTM

### Create word mask

In [15]:
WORDS = (tokens.punc == 0) & (tokens.num == 0)

### Extrct BOW from tokens

To extract a bag-of-words model from our tokens table, we apply a simple `groupby()` operation. Note that we can drop in our hyperparameters easily -- CHAPS and 'term_id' and be replaced. We can easily write a function to simplify this process and make it more configurable. 


In [16]:
BOW = tokens[WORDS].groupby(BAG + ['term_id'])['term_id'].count()

### Convert BOW to DTM

In [17]:
DTM = BOW.unstack().fillna(0)

### Create Bags table

The bags table stores the OHCO content for each doc, since we remove this from the DTM. We can add some stats to this table if we wanted to.

In [18]:
bags = pd.DataFrame(index = DTM.index)
# bags['term_count'] = DTM.sum(1)
# bags['tf'] = bags.term_count / bags.term_count.sum()

In [19]:
DTM = DTM.reset_index(drop=True)
DTM.index.name = 'bag_id'

## Compute Term Frequencies and Weights

### Compute TF

Note that TF is just the term count. It is often normalized in the computing the value, but it is defined as the count in the context of information retrieval.

### Compute IDF

In [20]:
N_docs = DTM.shape[0]
vocab['df'] = DTM[DTM > 0].count()
vocab['idf'] = np.log10(N_docs / vocab.df)

### Test: View most frequent non-stops by IDF

In [21]:
vocab[vocab.stop==0].sort_values('n', ascending=False).head(500)\
    .sort_values('idf', ascending=False).head(20)

,term_str,n,p,port_stem,stop,df,tf_sum,tf_mean,tf_max,tfidf_sum,tfidf_mean,tfidf_max,tfth_sum,tfth_mean,tfth_max,th_sum,th_mean,th_max,idf
term_id,,,,,,,,,,,,,,,,,,,
2383,bagman,231,0.000206,bagman,0,21,0.072401,0.000364,0.010866,0.234891,0.001180,0.035252,0.040429,0.000203,0.006067,0.558402,0.002806,0.070888,0.976634
12447,lockhart,251,0.000224,lockhart,0,22,0.096452,0.000485,0.018093,0.306447,0.001540,0.057486,0.065000,0.000327,0.012193,0.673907,0.003386,0.104732,0.956430
11940,kreacher,292,0.000260,kreacher,0,24,0.090647,0.000456,0.030331,0.276623,0.001390,0.092560,0.056611,0.000284,0.018942,0.624526,0.003138,0.152961,0.918642
18929,slughorn,406,0.000362,slughorn,0,26,0.128720,0.000647,0.023518,0.377945,0.001899,0.069053,0.115002,0.000578,0.021012,0.893427,0.004490,0.127234,0.883880
17305,riddle,297,0.000265,riddl,0,28,0.110134,0.000553,0.022198,0.311598,0.001566,0.062803,0.082083,0.000412,0.016544,0.745307,0.003745,0.121942,0.851695
5482,crouch,333,0.000297,crouch,0,29,0.105022,0.000528,0.015066,0.291818,0.001466,0.041862,0.080415,0.000404,0.011536,0.765694,0.003848,0.091187,0.836455
11942,krum,252,0.000225,krum,0,30,0.075605,0.000380,0.009716,0.206380,0.001037,0.026522,0.045813,0.000230,0.005887,0.605962,0.003045,0.064955,0.821732
6665,dobby,446,0.000397,dobbi,0,32,0.165526,0.000832,0.042866,0.436430,0.002193,0.113022,0.172670,0.000868,0.044716,1.043160,0.005242,0.194784,0.793703
8530,fleur,243,0.000217,fleur,0,34,0.077763,0.000391,0.007634,0.198231,0.000996,0.019459,0.050489,0.000254,0.004956,0.649270,0.003263,0.053690,0.767374


### Compute TFIDF

See [Simone Teufel's lectures](https://www.cl.cam.ac.uk/teaching/1415/InfoRtrv/lecture4.pdf)

```
TF: term count
N: number of docs
DF: number of docs with term
log = log10

(1 + log(TF)) * log( N / DF)
```

In [22]:
TFIDF = DTM * vocab['idf']

### Test: Stopwords Detected?

In [23]:
vocab[TFIDF.sum() == 0]

,term_str,n,p,port_stem,stop,df,tf_sum,tf_mean,tf_max,tfidf_sum,tfidf_mean,tfidf_max,tfth_sum,tfth_mean,tfth_max,th_sum,th_mean,th_max,idf
term_id,,,,,,,,,,,,,,,,,,,
1185,a,20951,0.018672,a,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1237,about,2501,0.002229,about,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1512,again,1878,0.001674,again,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1642,all,4324,0.003854,all,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1744,an,2634,0.002347,an,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1754,and,27433,0.024449,and,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2033,around,2184,0.001946,around,0,199,0.764945,0.003844,0.009487,0.0,0.0,0.0,4.611369,0.023173,0.057194,6.028367,0.030293,0.063753,0.0
2079,as,7605,0.006778,as,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2166,at,8652,0.007711,at,1,199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


### Add stats to Vocab

In [24]:
vocab['tfidf_sum'] = TFIDF.sum()
vocab['tfidf_mean'] = TFIDF.mean()
vocab['tfidf_max'] = TFIDF.max()

### Get Top words and Trim Matrix

Basically, implement this SQL query in Pandas:
```
SELECT * 
FROM vocab 
WHERE stop = 0
ORDER BY n DESC
LIMIT 1000
```

In [25]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1000):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)

### Remove proper nouns

These make it too easy to distinguish genres, as they have super high TFIDF values. 

In [26]:
proper_nouns = tokens.loc[tokens.pos == 'NNP', 'term_id'].unique()

In [27]:
top_n = 1000
# TOPV = get_top_terms(vocab, sort_col='n')
TOPV = get_top_terms(vocab.loc[~vocab.index.isin(proper_nouns)], sort_col='n')

### Create Reduced TFIDF matrix for later use

In [28]:
tfidf_small = TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'w'})

# Save data

In [29]:
with sqlite3.connect(db_name) as db:
    vocab.to_sql('vocab', db, if_exists='replace', index=True)
    tokens.to_sql('token', db, if_exists='replace', index=True)
    docs.to_sql('doc', db, if_exists='replace', index=True)
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)
    bags.reset_index().to_sql('bag', db, if_exists='replace', index=True, index_label='bag_id')

In [21]:
# END